## Lab 6
## Roshni Ram
## 201651045
## Implement the calculation of first and follow for any given grammar. Take grammar as user input.

In [1]:
n = int(input("Enter the number of production rules"))

Enter the number of production rules3


In [2]:
productions = []
for _ in range(n):
    productions.append(input())

E:E+T|T
T:T*F|F
F:(E)|id


### Remove left recursion

In [3]:
rules = []
some = {}
terminals = set()
for i in productions:
    rule = []
    flag = False
    sub_flag = False
    g = i.split(":")
    right = g[1].split("|")
    for k in right:
        sub = []
        if k[0]==g[0]:
            some[k[0]] = ""
            flag = True
            rule.append(g[0]+"\'"+":"+k[1:]+g[0]+"\'")
            some[g[0] + "'"] = ""
    if flag:
        rule.append(g[0]+"\'"+":#")
        for m in right:
            if m[0]!=g[0]:
                sub_flag = True
                rule.append(g[0]+":"+m+g[0]+"\'")
    if not sub_flag:
        rule.append(g[0]+":"+g[0]+"\'")
    if not flag:
        some[i[0]] = ""
        rules.append(i)
    if flag:
        rules.append(rule)

rules

[["E':+TE'", "E':#", "E:TE'"], ["T':*FT'", "T':#", "T:FT'"], 'F:(E)|id']

In [4]:
some


{'E': '', "E'": '', 'T': '', "T'": '', 'F': ''}

In [5]:
gram = set()
for rule in rules:
    if (isinstance(rule, list)):
        for i in range(len(rule)):
            startLetter = (rule[i].split(':'))[0]
            indices = [];
            for i in range(len(rule)):
                x = (rule[i].split(':'))[0]
                if(x == startLetter):
                    indices.append(i)
            a = rule[indices[0]]
            for k in range(1,len(indices)):
                a =a + "|" + rule[indices[k]][3:]
            gram.add(a)
    else:
        gram.add(str(rule))

In [6]:
gram = list(gram)
for prod in gram:
    if prod[1] == "'":
        some[prod[0:2]] = [prod[3:]]
    else:
        some[prod[0:1]] = [prod[2:]]

In [7]:
for key, value in some.items():
    b = "".join(value).split('|')
    valueList = []
    for i in range(len(b)):
        x = b[i]
        val = ()
        for j in range(len(x)):
            if x[j].islower():
                val = val + (x,)
                break
            elif(j+1<len(x) and x[j+1] == "'"):
                val = val + ((x[j] + x[j+1]),)
                break
            else:
                val += (x[j],)
        for var in val:
            if(not var[0].isupper()):
                terminals.add(var)
        valueList.append(val)
    some[key] = valueList

In [8]:
grammar = some
terminals = list(terminals)

In [9]:
nonterminals = []
for key,values in grammar.items():
    nonterminals.append(key)

In [10]:
def isTerminal(s):
    return s in terminals

In [11]:
def isNonTerminal(s):
    return s in nonterminals


In [12]:
def findSymbol(s):
    results = {}
    for nont in grammar:
        for production in grammar[nont]:
            for symbol in production:
                if s in production:
                    try:
                        results[nont]+=[production]
                    except KeyError:
                        results[nont]=[production]
    return results

In [13]:
def first(s):
    firsts = []
    for production in grammar[s]:
        if(isTerminal(production[0])):
            firsts.append(production[0])
        elif(isNonTerminal(production[0])):
            firsts+=first(production[0])
    return firsts

In [14]:
def follow(s):
    follows = []
    if s == next(iter(grammar)):
        follows.append("$")
    found = findSymbol(s)
    for nont in found:
        for production in found[nont]:
            if(production.index(s) < len(production)-1):
                nxt = production[production.index(s)+1]
                if(isTerminal(nxt)):
                    follows.append(nxt)
                elif(isNonTerminal(nxt)):
                    nxt = first(nxt)
                    if("#" in nxt):
                        nxt += follow(nont)
                    follows+= nxt
            else:
                if(s != nont):
                    follows = follow(nont)
    follows = list(set(follows))
    if("#" in follows):
        follows.remove("#")
    return follows

In [18]:
def printGrammar():
    for rule in grammar:
        str_rule = rule + " -> "
        for production in grammar[rule]:
            for symbol in production:
                str_rule+=symbol
            str_rule+="|"
        str_rule = str_rule[:-1]
        print(str_rule)

In [19]:

def printFirsts():
    print("First:")
    for nont in nonterminals:
        print(first(nont))

In [20]:
def printFollows():
    print("Follows:")
    for nont in nonterminals:
        print(follow(nont))

In [21]:
printGrammar()
print("\n")
printFirsts()
printFollows()

E -> TE'
E' -> +TE'|#
T -> FT'
T' -> *FT'|#
F -> (E)|id


First:
['(', 'id']
['+', '#']
['(', 'id']
['*', '#']
['(', 'id']
Follows:
[')', '$']
[')', '$']
[')', '+', '$']
[')', '+', '$']
[')', '*', '+', '$']
